# Duplicate Entities

This notebook is intended to test methods for detecting duplicate entities in planning data. It has been developed by focusing on conservation areas but should be easily extended to other entity types as it uses standard properties of the entities such as name and geometry.

The goal is to flag specific issues and then prioritise entities based on the number of issues  that are present.

In [ ]:
import urllib
from collections import defaultdict
from itertools import combinations

import geopandas as gpd
import numpy as np
import plotly.graph_objects as go
import polars as pl
import requests
from rtree import index
from shapely import MultiPolygon, overlaps
from shapely.wkt import loads

from data_quality_utils.polygon.plotting import plot_multipolygon
from data_quality_utils.polygon.utils import overlap_ratio, shortest_distance

In [ ]:
def plot_duplicates(duplicate_entities: list[MultiPolygon], fill_colours=None, zoom=15):
    fig = go.Figure()
    if fill_colours is None:
        fill_colours = [(255, 0, 0)] * len(duplicate_entities)
    for i, polygon in enumerate(duplicate_entities):
        fig = plot_multipolygon(
            polygon=polygon,
            fig=fig,
            name=f"{i+1}",
            line_color="black",
            fill_color=fill_colours[i],
            fill_alpha=0.3,
        )

    fig.update_layout(
        geo_scope="europe",
        map=dict(
            style="open-street-map",
            center=dict(lon=polygon.centroid.x, lat=polygon.centroid.y),
            zoom=zoom,
        ),
        showlegend=True,
        margin={"r": 100, "t": 50, "l": 100, "b": 50},
        height=800,
        width=1000,
    )
    return fig

Union-Find (also known as Disjoint Set Union or DSU) is a data structure used to efficiently group items into disjoint sets and check whether two items belong to the same group. It supports two key operations:

- find(x) — returns the root representative of the set containing x
- union(x, y) — merges the sets containing x and y

In grouping candidates Union-Find helps identify clusters of spatially nearby entities by merging entities whose geometries fall within a given distance (e.g. 100 meters). As entities are iterated over and found to be close to others, they’re merged into the same group. This enables fast grouping of related entities without redundant or nested checks, ensuring all spatially connected entities end up in the same cluster.

In [ ]:
class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, x):
        if self.parent.get(x, x) != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent.get(x, x)

    def union(self, x, y):
        self.parent[self.find(x)] = self.find(y)

## Data

From our search of `datasette`, there appears to be no publicly available endpoint for obtaining all entities in the planning data. Instead, we use the `conservation-area` dataset to get all conservation areas.

In [ ]:
# get organisation names
datasette_base_url = "https://datasette.planning.data.gov.uk/digital-land.csv"

query = """
select *
from organisation as o
"""
encoded_query = urllib.parse.urlencode({"sql": query})

r = requests.get(f"{datasette_base_url}?{encoded_query}", auth=("user", "pass"))

org_filename = "org_data.csv"
with open(org_filename, "wb") as f_out:
    f_out.write(r.content)

org = (
    pl.read_csv(org_filename)
    .select(["entity", "name"])
    .rename({"entity": "organisation_entity", "name": "organisation_name"})
)

In [ ]:
conservation_entities_url = "https://datasette.planning.data.gov.uk/conservation-area/entity.csv?_stream=on&_size=max"
r = requests.get(conservation_entities_url, auth=("user", "pass"))
conservation_df = pl.read_csv(r.content)
conservation_df = (
    conservation_df.with_columns(
        json=conservation_df["json"].str.json_decode(infer_schema_length=10000)
    )
    .unnest("json")
    .join(org, on="organisation_entity")
)

conservation_df = conservation_df.filter(pl.col("name") != "Polygon")

## Duplicate Checks

We do this in two parts. First we identify all potential duplicates and then we rank them by defining 'issues' and counting the number of issue each set of potential duplicates has. Issues in this context are facts about the entities that would be unusual if they are not duplicates - eg. a large amount of overlap between their geometries. 

### Identifying Candidates
The simplest check is whether two or more entities in the same region share the same name. We group the data by these columns and keep all entities that have two or more rows per group.

In [ ]:
name_candidate_df = (
    conservation_df.group_by("name")
    .agg(pl.len().alias("count"))
    .filter(pl.col("count") > 1)
    .with_row_index("group_id")
    .sort("count", descending=True)
)

candidate_df = conservation_df.join(
    name_candidate_df, on="name", how="left"
).with_columns(pl.col("group_id").fill_null(-1).cast(pl.Int32))
candidate_df

Another way is to create groups based on the proximity of their polygons

In [ ]:
# pick entities with non-null geometries
geometry_df = candidate_df.filter(pl.col("geometry").is_not_null()).select(
    ["entity", "geometry", "group_id"]
)
gdf = gpd.GeoDataFrame(
    geometry_df.to_dict(as_series=False),
    geometry=[loads(wkt) for wkt in geometry_df["geometry"].to_list()],
    crs="EPSG:4326",
).to_crs("EPSG:27700")

In [ ]:
entities = gdf["entity"].tolist()
geoms = gdf["geometry"].tolist()
group_ids = gdf["group_id"].to_list()

uf = UnionFind()

# create a spatial index for efficient search for nearby geometries
spatial_index = index.Index()
for idx, geom in enumerate(geoms):
    if geom and not geom.is_empty:
        spatial_index.insert(idx, geom.bounds)

# search for entities within 1000 metres
bbox_expand_meters = 100

for i, geom1 in enumerate(geoms):
    if not geom1 or geom1.is_empty:
        continue

    minx, miny, maxx, maxy = geom1.bounds
    expanded_bbox = (
        minx - bbox_expand_meters,
        miny - bbox_expand_meters,
        maxx + bbox_expand_meters,
        maxy + bbox_expand_meters,
    )

    nearby = list(spatial_index.intersection(expanded_bbox))
    for j in nearby:
        if i != j and geoms[j].distance(geom1) < bbox_expand_meters:
            uf.union(i, j)

# create clusters of entities based on name and proximity
clusters = defaultdict(list)
for i in range(len(entities)):
    root = uf.find(i)
    clusters[root].append(i)

next_group_id = max(group_ids) + 1 if group_ids else 0
cluster_id_map = {}

# assign group IDs
for root, members in clusters.items():
    existing_ids = [group_ids[i] for i in members if group_ids[i] >= 0]
    if existing_ids:
        assigned_id = min(existing_ids)
    else:
        assigned_id = next_group_id
        next_group_id += 1
    for i in members:
        cluster_id_map[i] = assigned_id

final_group_ids = [cluster_id_map.get(i, group_ids[i]) for i in range(len(entities))]

In [ ]:
geometry_candidate_df = pl.DataFrame({"entity": entities, "group_id": final_group_ids})
full_candidate_df = (
    candidate_df.join(
        geometry_candidate_df.rename({"group_id": "group_id_spatial"}),
        on="entity",
        how="left",
    )
    .with_columns(
        pl.when(
            pl.col("group_id_spatial").is_not_null() & (pl.col("group_id_spatial") >= 0)
        )
        .then(pl.col("group_id_spatial"))
        .otherwise(pl.col("group_id"))
        .alias("group_id")
    )
    .drop(["group_id_spatial"])
    .filter(pl.col("group_id") >= 0)
)
valid_group_ids = (
    full_candidate_df.group_by("group_id")
    .agg(pl.len().alias("count"))
    .filter(pl.col("count") >= 2)
)
full_candidate_df = full_candidate_df.join(valid_group_ids, on="group_id")
full_candidate_df

### Counting Issues

If we demand candidate duplicates share a name, we can then identify issues that may indicate that two entities are duplicates. 

We'll look for missing information, for fewer unique values like organisation name than one would expect for the number of entities with the same name, and missing or overlapping geometries. The general method here is that by counting the number of issues that are present for each candidate set of duplicates, we can prioritise them for investigation.

In [ ]:
def min_distance(pairs):
    distances = []
    for a, b in pairs:
        if a and b:
            distances.append(shortest_distance(a, b))
    if distances:
        return min(distances)
    else:
        return None


def max_overlap(pairs):
    areas = []
    for a, b in pairs:
        if a and b:
            areas.append(overlap_ratio(a, b))
    if areas:
        return max(areas)
    else:
        return None


def count_real_orgs(organisations):
    real_orgs = organisations.filter(~organisations.is_in([16, 600001]))
    return real_orgs.n_unique()

First generate some useful metrics that we expect are related to whether two or more entities with the same name are duplicates.

In [ ]:
records = []
for group_id, filtered_entities_df in full_candidate_df.group_by("group_id"):

    filtered_names = filtered_entities_df.get_column("name").to_list()

    possible_duplicates: list[MultiPolygon] = [
        loads(shape_str) for shape_str in filtered_entities_df["geometry"].to_list()
    ]
    candidate_pairs = list(combinations(possible_duplicates, 2))

    metrics = {
        "Group ID": group_id,
        "Names": filtered_names,
        "Number of names": len(filtered_names),
        "Number of candidates": len(possible_duplicates),
        "Number of organisations": count_real_orgs(
            filtered_entities_df["organisation_entity"]
        ),
        "Number of documentation-urls": filtered_entities_df["documentation-url"]
        .drop_nans()
        .n_unique(),
        "Number of document-urls": filtered_entities_df["document-url"]
        .drop_nans()
        .n_unique(),
        "Missing Geometries": filtered_entities_df["geometry"].is_null().sum(),
        "Max Overlap": max_overlap(candidate_pairs),
        "Number of Overlaps": sum([overlaps(a, b) for a, b in candidate_pairs]),
        "Smallest Distance": min_distance(candidate_pairs),
    }
    records.append(metrics)

Then go through and count the number of metrics that have values that indicate a problem.

In [ ]:
candidate_groups_df = pl.DataFrame(records)
candidate_groups_df = candidate_groups_df.with_columns(
    issue_count=(
        # fewer unique documents than candidates
        (
            candidate_groups_df["Number of document-urls"]
            < candidate_groups_df["Number of candidates"]
        )
        # fewer unique organisations than candidates
        + (
            candidate_groups_df["Number of candidates"]
            > candidate_groups_df["Number of organisations"]
        )
        # fewer unique documentation pages than candidates
        + (
            candidate_groups_df["Number of documentation-urls"]
            < candidate_groups_df["Number of candidates"]
        )
        # candidates have at least some overlap in geography
        + (candidate_groups_df["Max Overlap"].fill_null(0.0) > 0)
        # Candidates have more than 1 overlapping polygon
        + (candidate_groups_df["Number of Overlaps"] > 1)
        # The smallest distance between candidats is less than a km
        + (candidate_groups_df["Smallest Distance"].fill_null(100) < 1)
    )
)

In [ ]:
(
    candidate_groups_df.filter(
        (pl.col("issue_count") > 0) & (pl.col("Number of candidates") <= 10)
    ).sort(by="issue_count", descending=True)
)

# Results

## Overview

- We find 986 groups of entities with the same name, 976 have at least one issue
- We find additional 381 groups based on proximity
- Out of 1365 total groups there are 1214 with at least one issue
- High issue counts do seem to point to clear duplicates
- Overlap can be used to find obvious duplicates
- Missing/incomplete data is a major problem that should be resolved before entity de-duplication
- Grouping by proximity can include distinct entities in the same group, but it also finds clear duplicates with slight name variations


## Case Studies

### Fragmented Entities
Wymondham Conservation Area is typical of the high issue count candidates. There are 13 entities that share a name, documentation-url and document-url. However, there are 13 unique geometries in a set of 13 "duplicates". These 13 areas form one contiguous area which is documented just once on the South Norfolk District Council website. Wymondham is specifically listed as a single conservation area on their website so it seems this is a single entity where the polygons that define it have been logged separately.

Several other top listed candidates (Diss, Wingham, Petersfield) have the same issue though Petersfield has the additional issue of one entity that covers the whole area plus 8 entities that record each of its sections.

In [ ]:
name = "Wymondham Conservation Area"
possible_duplicates: list[MultiPolygon] = [
    loads(shape_str)
    for shape_str in conservation_df.filter(pl.col("name") == name)[
        "geometry"
    ].to_list()
    if shape_str
]
colours = [
    (np.random.randint(0, 255), 0, np.random.randint(0, 255))
    for i in range(len(possible_duplicates))
]
fig = plot_duplicates(possible_duplicates, fill_colours=colours, zoom=13)
fig.show()

### Complete Overlap

Sorting instead by the amount of overlap between polygons, we get entities that are clear duplicates. For conservation areas like Stone, we get a small number of duplicate entities with essentially identical geometries. For more complex cases like Farringdon, we get one overarching polygon with several other entities that completely duplicate sections of it, leading to high overlaps.

In [ ]:
name = "Stone"
possible_duplicates: list[MultiPolygon] = [
    loads(shape_str)
    for shape_str in conservation_df.filter(pl.col("name") == name)[
        "geometry"
    ].to_list()
    if shape_str
]

colours = [(120, 120, 120), (255, 0, 0), (0, 0, 255)]

fig = plot_duplicates(possible_duplicates, fill_colours=colours, zoom=15)
fig.show()

### Entities with name variations

Using only name to identify candidate groups can miss out on entities with very close/overlapping polygons but slightly different names. An example is Pulham Market, which has 3 entities listed under "Pulham Market Conservation Area" belonging to South Norfolk District Council, and one under "Pulham Market" belonging to Historic England.

In [ ]:
(
    conservation_df.filter(pl.col("name").str.contains("Pulham Market"))
    .group_by("organisation_name")
    .agg(pl.col("name"))
)

In [ ]:
name = "Pulham Market"
possible_duplicates: list[MultiPolygon] = [
    loads(shape_str)
    for shape_str in conservation_df.filter(pl.col("name").str.contains(name))[
        "geometry"
    ].to_list()
    if shape_str
]

colours = [
    (np.random.randint(0, 255), 0, np.random.randint(0, 255))
    for i in range(len(possible_duplicates))
]

fig = plot_duplicates(possible_duplicates, fill_colours=colours, zoom=15)
fig.show()

### Entities with incorrect names

Some entities are not listed under their correct names and instead use code names or other identifiers. In the case of Abinger Hammer we have 3 overlapping polygons, one covering the whole area and 2 smaller ones covering parts. However the two with correct names are listed under Historic England whereas the unnamed one correctly belongs to Mole Valley District Council. Identifying candidates only by name would not flag this case, but using polygon distance both
- identifies duplicates,
- resolves names and
- resolves organisation entities

In [ ]:
entity_ids = [44002548, 44004253, 44008695]
(
    conservation_df.filter(pl.col("entity").is_in(entity_ids))
    .group_by("organisation_name")
    .agg(pl.col("name"), pl.col("entity"))
)

In [ ]:
entity_ids = [44002548, 44004253, 44008695]
possible_duplicates: list[MultiPolygon] = [
    loads(shape_str)
    for shape_str in conservation_df.filter(pl.col("entity").is_in(entity_ids))[
        "geometry"
    ].to_list()
    if shape_str
]

colours = [
    (np.random.randint(0, 255), 0, np.random.randint(0, 255))
    for i in range(len(possible_duplicates))
]

fig = plot_duplicates(possible_duplicates, fill_colours=colours, zoom=15)
fig.show()

### Entities with name variation, under different organisations and with missing geometries

The duplicate candidate finder identified a group of 5 entities listed under different names: Salhouse, Salhouse Conservation Area, Salhouse BA Conservation Area, listed under Broadland District Council, MHCLG or Historic England. Their polygons are completely overlapping which makes them a clear duplicate.

In [ ]:
name = "Salhouse"
(
    conservation_df.filter(pl.col("name").str.contains(name))
    .group_by("organisation_name")
    .agg(pl.col("name"))
)

In [ ]:
name = "Salhouse"
possible_duplicates: list[MultiPolygon] = [
    loads(shape_str)
    for shape_str in conservation_df.filter(pl.col("name").str.contains(name))[
        "geometry"
    ].to_list()
    if shape_str
]

colours = [
    (np.random.randint(0, 255), 0, np.random.randint(0, 255))
    for i in range(len(possible_duplicates))
]

fig = plot_duplicates(possible_duplicates, fill_colours=colours, zoom=15)
fig.show()

### Missing Data

Another common failure mode is when two entities have the same name but should be under different organiations. For example Poulton which appears to be the name of a place in the Cotswolds as well as Cheshire but both have been assigned to MHCLG (entity 600001) instead of their local council. 

These cases are also often missing geometries but do have documentation-urls. If some of the previous validation steps were run documentation-url and document-url might be fixed. A simple rule to validate the organisation given the documentation-url might fix the assignment to MHCLG. Geometry might be obtained if the organisation leads to the correct endpoint or if AI tools are used to extract polygons from documents.

In [ ]:
name = "Poulton"
conservation_df.filter(pl.col("name") == name)

In [ ]:
name = "Tiverton"
conservation_df.filter(pl.col("name") == name)

In [ ]:
name = "Moor Park"
conservation_df.filter(pl.col("name") == name)